# Normalize String Characters

ASCII transliterations of Unicode text:
[Unidecode](https://pypi.org/project/Unidecode/)

In [1]:
import collections
import unicodedata

import pandas as pd

## Functions to decode/normalize string characters

In [2]:
def normalize_str(s):
    """ Strip accents from input String. """
    normalized = []
   
    for c in unicodedata.normalize('NFD', s):
        if unicodedata.category(c) != 'Mn':
            normalized.append(c)
    return ''.join(normalized)

or the short version

In [3]:
def normalize_str(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)  if unicodedata.category(c) != 'Mn')

### Other versions

```python
def _normalize_str(s):
    """ Strip accents from input String. """

    try:
        s = unicode(s, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    s = unicodedata.normalize('NFD', s)
    s = s.encode('ascii', 'ignore')
    s = s.decode("utf-8")
    return str(s)
```

using in a pandas dataframe

```python
df["column_to_convert"] = (
    df["column_to_convert"]
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
)
```

## Decode the latin characters

### download table of latin [graphemes](Grapheme)

In [4]:
df = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_Latin-script_letters',
    attrs={'class':'wikitable'}
)

#### Basic latin characters [&#x1F517;](https://en.wikipedia.org/wiki/List_of_Latin-script_letters#Basic_Latin)

In [5]:
raw_base = df[0]
raw_base.head()

0   1   2   3   4   5   6   7   8   9   ...  16  17  18  19  20  21  22  \
0  Aa  Bb  Cc  Dd  Ee  Ff  Gg  Hh  Ii  Jj  ...  Qq  Rr  Ss  Tt  Uu  Vv  Ww   

   23  24  25  
0  Xx  Yy  Zz  

[1 rows x 26 columns]

In [6]:
base = raw_base.T.values.reshape(1, 26).tolist()[0]
base = sum(map(list, base), [])
' '.join(base)

'A a B b C c D d E e F f G g H h I i J j K k L l M m N n O o P p Q q R r S s T t U u V v W w X x Y y Z z'

#### Characteres extensions [&#x1F517;](https://en.wikipedia.org/wiki/List_of_Latin-script_letters#Extensions)

In [7]:
raw_extensions = df[1]
raw_extensions.head()

Letter              Name                                              Notes
0      ᴀ   Small capital A  Nonstandard phonetic symbol; Uralic Phonetic A...
1  Ɐ ɐ ᵄ          Turned A                         Near-open central vowel[3]
2  Ɑ ɑ ᵅ  Alpha (script A)  IPA (open back unrounded vowel); Duka, Fe'fe, ...
3      ꬰ      Barred alpha  Otto Bremer's phonetic transcription system fo...
4      ꭤ    Inverted alpha                           Americanist phonetics[5]

In [8]:
extensions = raw_extensions['Letter'].str.split(' ').values.tolist()
extensions = sum(extensions, [])
extensions[:10]

['ᴀ', 'Ɐ', 'ɐ', 'ᵄ', 'Ɑ', 'ɑ', 'ᵅ', 'ꬰ', 'ꭤ', 'Ɒ']

#### Letters with diacritics [&#x1F517;](https://en.wikipedia.org/wiki/List_of_Latin-script_letters#Letters_with_diacritics)

In [9]:
raw_diacritics = df[2]
raw_diacritics.head()

Letter                         Name  \
0      ẚ       A with right half ring   
1    À à                 A with grave   
2    Á á                 A with acute   
3    Â â            A with circumflex   
4    Ầ ầ  A with circumflex and grave   

                                               Notes  
0                                                NaN  
1  Aghem, Ahlon, Arammba, Awing, Baka, Bali (Adam...  
2  Afrikaans, Aghem, Ahlon, Arammba, Awing, Bafia...  
3  Awing, Bangolan, Berber, Dutch, Emilian, Ewond...  
4                                         Vietnamese

In [10]:
diacritics = raw_diacritics['Letter'].str.split(' ').values.tolist()
diacritics = sum(diacritics, [])
diacritics[:10]

['ẚ', 'À', 'à', 'Á', 'á', 'Â', 'â', 'Ầ', 'ầ', 'Ấ']

#### Ligatures [&#x1F517;](https://en.wikipedia.org/wiki/List_of_Latin-script_letters#Ligatures)

In [11]:
raw_ligatures = df[3]
raw_ligatures.head()

Letter                  Name  \
0  Ꜳ ꜳ 𐞀                    AA   
1  Ꜳ́ ꜳ́         AA with acute   
2  Ꜳ̋ ꜳ̋  AA with double acute   
3  Ꜳ̇ ꜳ̇     AA with dot above   
4  Ꜳ̈ ꜳ̈     AA with diaeresis   

                                               Notes  
0  Medieval Nordic vowel /aː/;[9] Superscript for...  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4                                                NaN

In [12]:
ligatures = raw_ligatures['Letter'].str.split(' ').values.tolist()
ligatures = sum(ligatures, [])
ligatures[:10]

['Ꜳ', 'ꜳ', '𐞀', 'Ꜳ́', 'ꜳ́', 'Ꜳ̋', 'ꜳ̋', 'Ꜳ̇', 'ꜳ̇', 'Ꜳ̈']

### decode all characters

In [13]:
to_decode = base + extensions + diacritics + ligatures
print('Number of characters', len(to_decode))

Number of characters 4064


In [14]:
decoded = collections.defaultdict(list)

for char in to_decode:
    _decoded = normalize_str(char)
    decoded[_decoded].append(char)
    
decoded = dict(decoded)

In [15]:
for k, v in decoded.items():
    print('-- ', k, ' -- ')
    print(' '.join(v), '\n')

--  A  -- 
A À Á Â Ầ Ấ Ẫ Ẩ Ã Ã̀ Ã́ Ã̂ Ã̌ Ã̍ Ã̎ Ā Ā̀ Ā́ Ā̂ Ā̃ Ā̃́ Ā̄ Ā̆ Ā̆́ Ā̈ Ā̊ Ā̌ Ă Ằ Ắ Ẵ Ẳ Ȧ Ȧ́ Ǡ Ä Ä́ Ä̀ Ä̂ Ä̃ Ǟ Ǟ̆ Ä̆ Ä̌ Ả Å Å Ǻ Å̂ Å̃ Å̄ Å̄̆ Å̆ A̋ Ǎ A̍ A̎ Ȁ Ȃ A̐ A̓ A̧ À̧ Á̧ Â̧ Ǎ̧ A̭ A̰ À̰ Á̰ Ā̰ Ä̰ Ä̰́ Ą Ą̀ Ą́ Ą̂ Ą̃ Ą̄ Ą̄̀ Ą̄́ Ą̄̂ Ą̄̌ Ą̇ Ą̈ Ą̈̀ Ą̈́ Ą̈̂ Ą̈̌ Ą̈̄ Ą̊ Ą̌ Ą̋ Ą̱ Ą̱̀ Ą̱́ A᷎ A̱ À̱ Á̱ Â̱ Ã̱ Ā̱ Ā̱̀ Ā̱́ Ā̱̂ Ä̱ Ä̱̀ Ä̱́ Ä̱̂ Ä̱̌ Å̱ Ǎ̱ A̱̥ Ạ Ạ́ Ạ̀ Ậ Ạ̃ Ạ̄ Ặ Ạ̈ Ạ̈̀ Ạ̈́ Ạ̈̂ Ạ̈̌ Ạ̌ Ạ̍ A̤ À̤ Á̤ Â̤ Ä̤ Ḁ Ḁ̂ Ḁ̈ A̯ A̩ À̩ Á̩ Â̩ Ã̩ Ā̩ Ǎ̩ A̩̍ A̩̓ A͔ Ā͔ 

--  a  -- 
a à á â ầ ấ ẫ ẩ ã ã̀ ã́ ã̂ ã̌ ã̍ ã̎ ā ā̀ ā́ ā̂ ā̃ ā̃́ ā̄ ā̆ ā̆́ ā̈ ā̊ ā̌ ă ằ ắ ẵ ẳ ȧ ȧ́ ǡ ä ä́ ä̀ ä̂ ä̃ ǟ ǟ̆ ä̆ ä̌ ả å ǻ å̂ å̃ å̄ å̄̆ å̆ a̋ ǎ a̍ a̎ ȁ ȃ a̐ a̓ a̧ à̧ á̧ â̧ ǎ̧ a̭ a̰ à̰ á̰ ā̰ ä̰ ä̰́ ą ą̀ ą́ ą̂ ą̃ ą̄ ą̄̀ ą̄́ ą̄̂ ą̄̌ ą̇ ą̈ ą̈̀ ą̈́ ą̈̂ ą̈̌ ą̈̄ ą̊ ą̌ ą̋ ą̱ ą̱̀ ą̱́ a᷎ a̱ à̱ á̱ â̱ ã̱ ā̱ ā̱̀ ā̱́ ā̱̂ ä̱ ä̱̀ ä̱́ ä̱̂ ä̱̌ å̱ ǎ̱ a̱̥ ạ ạ́ ạ̀ ậ ạ̃ ạ̄ ặ ạ̈ ạ̈̀ ạ̈́ ạ̈̂ ạ̈̌ ạ̌ ạ̍ a̤ à̤ á̤ â̤ ä̤ ḁ ḁ̂ ḁ̈ a̯ a̩ à̩ á̩ â̩ ã̩ ā̩ ǎ̩ a̩̍ a̩̓ a͔ ā͔ 

--  B  -- 
B B̀ B́ B̂ B̃ B̄ Ḃ B̈ B̒ B̕ Ḇ Ḇ̂ Ḅ B̤ B̥ B̬ B̪ 

--  b  -- 
b b̀ b́ b̂ b̃ b̄ ḃ b̈